In [ ]:
%pip install codeflare-sdk -U

In [1]:
import os, sys, yaml
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication
from codeflare_sdk.job import RayJobClient
from time import sleep

In [ ]:
%pip show codeflare-sdk

In [ ]:
#parameters
namespace = "default"
openshift_api_url = "has to be specified"
kubernetes_user_bearer_token = "has to be specified"
num_gpus = "has to be specified"
s3_bucket_name = "has to be specified"
s3_access_key_id = "has to be specified"
s3_secret_access_key = "has to be specified"
s3_default_region = "has to be specified"

In [2]:
# Create the training and evaluation datasets.
# This can be run only once.
!{sys.executable} -m pip install datasets
create_dataset_path=os.path.join(os.getcwd(),'../notebooks')
if os.path.exists(create_dataset_path):
    sys.path.append(os.path.abspath(create_dataset_path))
import create_dataset
create_dataset.main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 112.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
auth = TokenAuthentication(
    token=kubernetes_user_bearer_token,
    server=openshift_api_url,
    skip_tls=True,
)
auth.login()

In [28]:
# Configure the Ray cluster
cluster = Cluster(ClusterConfiguration(
    name='ray',
    namespace=namespace,
    head_cpus=1,
    head_memory=4,
    head_gpus=0,
    num_workers=1,
    min_cpus=1,
    max_cpus=1,
    min_memory=1,
    max_memory=4,
    num_gpus=int(num_gpus),
    write_to_file=True,
    verify_tls=False
))
print("raycluster config created","*"*8)

Written to: /opt/app-root/src/.codeflare/resources/ray.yaml


In [29]:
import yaml 
cluster_yaml = None
directory_path = os.path.expanduser("~/.codeflare/resources/")
outfile = os.path.join(directory_path, "ray.yaml")
with open(outfile, "r") as f:
    cluster_yaml = yaml.load(f, yaml.FullLoader)
    "# Add toleration for GPU nodes to Ray cluster worker podn",
    cluster_yaml["spec"]["workerGroupSpecs"][0]["template"]["spec"]["tolerations"]=[{"key": "nvidia.com/gpu", "value": "NONE", "effect": "NoSchedule"}]
    try:
        del cluster_yaml["metadata"]["labels"]["kueue.x-k8s.io/queue-name"]
        print("Kueue label deleted")
    except Exception as e:
        print(e)
    with open(outfile, "w") as f:
        yaml.dump(cluster_yaml, f, default_flow_style=False)

Kueue label deleted


In [30]:
# Create the Ray cluster
cluster.up()
# Wait until status is updated
cluster.wait_ready()

In [32]:
cluster.details()

                         🚀 CodeFlare Cluster Details 🚀                         
                                                                                 
 ╭─────────────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                                      │ 
 │   ray                                                           Active ✅   │ 
 │                                                                             │ 
 │   URI: ray://ray-head-svc.ray-finetune-llm-deepspeed.svc:10001              │ 
 │                                                                             │ 
 │   ]8;id=983341;https://ray-dashboard-ray-ray-finetune-llm-deepspeed.apps.o1i1y2s4h9x4b9p.pdxp.p1.openshiftapps.com\Dashboard🔗]8;;\                                                               │ 
 │                                                                             │ 
 │                       Cluster Resources                                     │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮                 │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │                 │ 
 │   │             │  │                                      │                 │ 
 │   │  1          │  │  48G~48G     8           1           │                 │ 
 │   │             │  │                                      │                 │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯                 │ 
 ╰─────────────────────────────────────────────────────────────────────────────╯

RayCluster(name='ray', status=<RayClusterStatus.READY: 'ready'>, head_cpus=16, head_mem='48G', head_gpu=0, workers=1, worker_mem_min='48G', worker_mem_max='48G', worker_cpu=8, worker_gpu=1, namespace='ray-finetune-llm-deepspeed', dashboard='https://ray-dashboard-ray-ray-finetune-llm-deepspeed.apps.o1i1y2s4h9x4b9p.pdxp.p1.openshiftapps.com')

In [34]:
# Initialize the Job Submission Client
client = cluster.job_client

In [35]:
# The S3 bucket where to store checkpoint.
# It can be set manually, otherwise it's retrieved from configured the data connection.
s3_bucket = s3_bucket_name
if not s3_bucket_name and s3_bucket_name!=None and s3_bucket_name!="":
    s3_bucket = os.environ.get('AWS_S3_BUCKET')
assert s3_bucket, "An S3 bucket must be provided to store checkpoints"

In [ ]:
submission_id = client.submit_job(
    entrypoint="python ray_finetune_llm_deepspeed.py "
               "--model-name=meta-llama/Llama-2-7b-chat-hf "
               "--lora "
               "--num-devices=1 "
               "--num-epochs=3 "
               "--ds-config=./deepspeed_configs/zero_3_llama_2_7b.json "
               f"--storage-path=s3://{s3_bucket_name}/ray-finetune-llm-deepspeed3/"
               "--batch-size-per-device=32 "
               "--eval-batch-size-per-device=32 "
               "--as-test ",
    runtime_env={
        "env_vars": {
            "AWS_ACCESS_KEY_ID": s3_access_key_id,
            "AWS_SECRET_ACCESS_KEY": s3_secret_access_key,
            "AWS_DEFAULT_REGION": s3_default_region,
        },
        "pip": "requirements.txt",
        "working_dir": "./",
        "excludes": ["/docs/", "*.ipynb", "*.md"]
    },
)
print(submission_id)

2024-07-26 10:57:35,471	INFO dashboard_sdk.py:385 -- Package gcs://_ray_pkg_e1883352aefb99c4.zip already exists, skipping upload.


raysubmit_Hmn8GnNcSa1UJFaC


In [37]:
client.stop_job(submission_id)

True

In [42]:
cluster.down()